In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from collections import Counter
import numpy as np
import os
from PIL import Image
import random

import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset, Subset
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split

In [2]:
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_value = 42
set_seed(seed_value)

In [3]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.image_paths = []
        self.labels = []

        for cls_name in self.classes:
            cls_path = os.path.join(root_dir, cls_name)
            for img_name in os.listdir(cls_path):
                img_path = os.path.join(cls_path, img_name)
                self.image_paths.append(img_path)
                self.labels.append(self.class_to_idx[cls_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')  

        if self.transform:
            image = self.transform(image)

        return image, label

data_dir = '/kaggle/input/test-dataset/Fire-Detection' 

image_size = 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

full_dataset = CustomImageDataset(root_dir=data_dir, transform=transform)

labels = [label for _, label in full_dataset]
class_counts = Counter(labels)
print('Class Distribution:', class_counts)

Class Distribution: Counter({0: 541, 1: 110})


In [4]:
train_idx, remaining_idx = train_test_split(
    range(len(full_dataset)),
    test_size=0.2,
    stratify=full_dataset.labels,
    random_state=42  
)

val_idx, test_idx = train_test_split(
    remaining_idx,
    test_size=0.5,
    stratify=[full_dataset.labels[i] for i in remaining_idx],
    random_state=42
)

train_dataset = Subset(full_dataset, train_idx)
val_dataset = Subset(full_dataset, val_idx)
test_dataset = Subset(full_dataset, test_idx)

print('Class Distribution of Training Set:', Counter([full_dataset.labels[i] for i in train_idx]))
print('Validation Set Class Distribution:', Counter([full_dataset.labels[i] for i in val_idx]))
print('Test Set Class Distribution:', Counter([full_dataset.labels[i] for i in test_idx]))

batch_size = 32  

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print('\nDataLoaders created.')
print('Training DataLoader size:', len(train_loader))
print('Validation DataLoader size:', len(val_loader))
print('Test DataLoader size:', len(test_loader))

Class Distribution of Training Set: Counter({0: 432, 1: 88})
Validation Set Class Distribution: Counter({0: 54, 1: 11})
Test Set Class Distribution: Counter({0: 55, 1: 11})

DataLoaders created.
Training DataLoader size: 17
Validation DataLoader size: 3
Test DataLoader size: 3


In [5]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2) 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print('The pre-trained ResNet18 model was loaded and its last layer was updated.')
print(model)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 151MB/s]


The pre-trained ResNet18 model was loaded and its last layer was updated.
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, 

In [6]:
criterion = nn.BCEWithLogitsLoss()
learning_rate = 0.001  
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print('The loss function and optimization algorithm are defined.')

The loss function and optimization algorithm are defined.


In [7]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, patience=5):
    best_val_loss = float('inf')
    epochs_no_improve = 0
    history = {'train_loss': [], 'val_loss': [], 'val_accuracy': []}

    for epoch in range(num_epochs):
        model.train()  
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device).float().unsqueeze(1) 
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs[:, 0].unsqueeze(1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        history['train_loss'].append(epoch_loss)

        model.eval()  
        val_loss = 0.0
        correct_predictions_val = 0
        total_samples_val = 0

        with torch.no_grad():  
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device).float().unsqueeze(1)
                outputs = model(inputs)
                outputs = outputs[:, 0].unsqueeze(1)
                val_loss += criterion(outputs, labels).item() * inputs.size(0)
                probabilities = torch.sigmoid(outputs)
                predictions = (probabilities > 0.5).float()
                correct_predictions_val += (predictions == labels).sum().item()
                total_samples_val += labels.size(0)

        epoch_val_loss = val_loss / len(val_loader.dataset)
        epoch_val_accuracy = correct_predictions_val / total_samples_val
        history['val_loss'].append(epoch_val_loss)
        history['val_accuracy'].append(epoch_val_accuracy)

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss of Train: {epoch_loss:.4f}, Validation Loss:: {epoch_val_loss:.4f}, Validation Accuracy: {epoch_val_accuracy:.4f}')

        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), '/kaggle/working/best_model.pth') 
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print(f'Early stopping applied! Best validation loss: {best_val_loss:.4f}')
                break

    model.load_state_dict(torch.load('/kaggle/working/best_model.pth'))
    return model, history

num_epochs = 20  
patience = 5    
trained_model, history = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, patience)

Epoch [1/20], Loss of Train: 0.2714, Validation Loss:: 3.4018, Validation Accuracy: 0.6769
Epoch [2/20], Loss of Train: 0.1415, Validation Loss:: 0.1272, Validation Accuracy: 0.9692
Epoch [3/20], Loss of Train: 0.0961, Validation Loss:: 0.2138, Validation Accuracy: 0.9538
Epoch [4/20], Loss of Train: 0.0624, Validation Loss:: 0.2293, Validation Accuracy: 0.9231
Epoch [5/20], Loss of Train: 0.0422, Validation Loss:: 0.1347, Validation Accuracy: 0.9385
Epoch [6/20], Loss of Train: 0.0563, Validation Loss:: 0.1210, Validation Accuracy: 0.9692
Epoch [7/20], Loss of Train: 0.0439, Validation Loss:: 0.2288, Validation Accuracy: 0.9385
Epoch [8/20], Loss of Train: 0.0629, Validation Loss:: 0.2061, Validation Accuracy: 0.9538
Epoch [9/20], Loss of Train: 0.0982, Validation Loss:: 0.1208, Validation Accuracy: 0.9538
Epoch [10/20], Loss of Train: 0.0557, Validation Loss:: 0.3949, Validation Accuracy: 0.8769
Epoch [11/20], Loss of Train: 0.0489, Validation Loss:: 0.2544, Validation Accuracy: 0.95

In [8]:
def evaluate_model(model, test_loader, criterion):
    model.eval()  
    test_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    all_predictions = []
    all_labels = []
    all_probabilities = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device).float().unsqueeze(1)
            outputs = model(inputs)
            outputs = outputs[:, 0].unsqueeze(1)
            test_loss += criterion(outputs, labels).item() * inputs.size(0)
            probabilities = torch.sigmoid(outputs)
            predictions = (probabilities > 0.5).float()
            correct_predictions += (predictions == labels).sum().item()
            total_samples += labels.size(0)
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())

    avg_test_loss = test_loss / len(test_loader.dataset)
    test_accuracy = correct_predictions / total_samples

    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

    from sklearn.metrics import classification_report, confusion_matrix
    print('\nClassification Report:')
    print(classification_report(all_labels, all_predictions, target_names=['No Fire', 'Fire']))

    print('\nConfusion Matrix:')
    print(confusion_matrix(all_labels, all_predictions))

evaluate_model(trained_model, test_loader, criterion)

Test Loss: 0.1035, Test Accuracy: 0.9697

Classification Report:
              precision    recall  f1-score   support

     No Fire       1.00      0.96      0.98        55
        Fire       0.85      1.00      0.92        11

    accuracy                           0.97        66
   macro avg       0.92      0.98      0.95        66
weighted avg       0.97      0.97      0.97        66


Confusion Matrix:
[[53  2]
 [ 0 11]]
